## import libraries

In [1]:
# Hide deprecation, future, and user warnings
import warnings
ignore_warnings = lambda category: warnings.filterwarnings("ignore", category=category)
ignore_warnings(DeprecationWarning)
ignore_warnings(UserWarning)
ignore_warnings(FutureWarning)

# Import libraries
import kaggle
import os
import pandas as pd

# Import project-specific environment variables
from dotenv import load_dotenv
load_dotenv()

# Format dataframes to show commas
pd.options.display.float_format = '{:,.0f}'.format

## Download the dataset from Kaggle

In [2]:
def download_dataset(download: bool = False) -> str:
    """Download the data set from Kaggle
    Args:
        download (bool, optional): Flag to download the dataset. Defaults to False.
    Returns:
        None
    """
    fname = 'PRODUCT SALES.csv'
    if os.path.exists(fname) or not download:
        print('Skipping download as the file already exists')
    else:
        print('Starting download of the customer segmentation data from Kaggle')
        kaggle.api.dataset_download_files(
            dataset='kanyianalyst/customer-age-group-segmentation',
            path='.',
            force=True,
            quiet=False,
            unzip=True
        )
    return fname

## Load the file into local memory
### Note: the age group id, month, day, and year columns contain data that don't match our expectations. For example:
- There are 4 unique age groups, but only 1 age group NUMBER provided in the raw data
- All month values = January
### We'll replace these columns when loading the file

In [3]:
def load_file(fname: str) -> pd.DataFrame:
    """Load the csv file into local memory, rename the headers, and
    optjimise the data types.
    
    Args:
        fname (str): The name of the customer segmentation file
        
    Returns:
        pd.DataFrame: The customer segmentation data
    """
    header = [
        'date', 'day', 'month', 'year', 'customer_age', 'age_group',
        'age_group_id', 'customer_gender', 'country', 'product_category',
        'order_quantity', 'unit_cost', 'unit_price', 'profit', 'cost',
        'revenue'
    ]
    dtype_map = dict(customer_age='int8',
                     order_quantity='int16',
                     unit_cost='float32',
                     unit_price='float32',
                     profit=float,
                     cost=float,
                     revenue=float)
    df = pd.read_csv(fname,
                     sep=',',
                     header=0,
                     names=header,
                     dtype=dtype_map,
                     date_format='%d-%m-%Y',
                     parse_dates=['date'])
    df[['age_group', 'customer_gender', 'country', 'product_category']] = df[[
        'age_group', 'customer_gender', 'country', 'product_category'
    ]].astype('category')
    
    # Drop columns producing data inconsistent with our expectations
    df.drop(columns=['age_group_id', 'day', 'month', 'year'], inplace=True)
    
    # Codify categoricals
    df['age_group_id'] = df['age_group'].cat.codes
    df['customer_gender_id'] = df['customer_gender'].cat.codes
    df['product_category_id'] = df['product_category'].cat.codes
    
    # Fix the capitalisation for Country
    df['country'] = df['country'].str.title()
    
    # Recreating date columns since the original dataset is incorrect
    df['month'] = pd.to_datetime(df['date']).dt.month
    df['day'] = pd.to_datetime(df['date']).dt.day
    df['year'] = pd.to_datetime(df['date']).dt.year
    return df

## Load the file into local memory

In [4]:
fname = download_dataset()
df = load_file(fname)
df.info()

Skipping download as the file already exists
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113036 entries, 0 to 113035
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   date                 113036 non-null  object  
 1   customer_age         113036 non-null  int8    
 2   age_group            113036 non-null  category
 3   customer_gender      113036 non-null  category
 4   country              113036 non-null  object  
 5   product_category     113036 non-null  category
 6   order_quantity       113036 non-null  int16   
 7   unit_cost            113036 non-null  float32 
 8   unit_price           113036 non-null  float32 
 9   profit               113036 non-null  float64 
 10  cost                 113036 non-null  float64 
 11  revenue              113036 non-null  float64 
 12  age_group_id         113036 non-null  int8    
 13  customer_gender_id   113036 non-null  int8    
 14  product

### Reduced the file size 3+ MB

## View first 5 rows

In [5]:
df.head()

,date,customer_age,age_group,customer_gender,country,product_category,order_quantity,unit_cost,unit_price,profit,cost,revenue,age_group_id,customer_gender_id,product_category_id,month,day,year
0,01-01-17,17,Youth (<25),M,Canada,Bikes,2,"1,519","2,443","1,848","3,038","4,886",3,1,1,1,1,2017
1,01-01-17,23,Youth (<25),M,Australia,Bikes,2,"1,252","2,295","2,086","2,504","4,590",3,1,1,1,1,2017
2,01-01-17,33,Young Adults (25-34),F,France,Bikes,2,"2,171","3,578","2,814","4,342","7,156",2,0,1,1,1,2017
3,01-01-17,39,Adults (35-64),M,United States,Bikes,2,713,"1,120",814,"1,426","2,240",0,1,1,1,1,2017
4,01-01-17,42,Adults (35-64),M,United States,Bikes,2,344,540,392,688,"1,080",0,1,1,1,1,2017


## Describe numericals

In [6]:
df[['customer_age', 'order_quantity', 'unit_cost', 'unit_price', 'profit', 'cost', 'revenue']].describe()

,customer_age,order_quantity,unit_cost,unit_price,profit,cost,revenue
count,"113,036","113,036","113,036","113,036","113,036","113,036","113,036"
mean,36,12,267,453,373,469,842
std,11,10,550,922,595,885,"1,466"
min,17,1,1,2,1,1,2
25%,28,2,2,5,39,28,70
50%,35,10,9,24,135,108,245
75%,43,20,42,70,432,432,880
max,87,32,"2,171","3,578","26,158","42,978","69,136"


## Segment analysis

In [7]:
df.query("country in ['France', 'United States', 'United Kingdom'] & product_category == 'Accessories'")\
    .pivot_table(
        index='age_group', 
        columns=['product_category', 'country'], 
        values='revenue', 
        aggfunc='sum',
        fill_value=0,
        observed=True, 
        sort=True
)

product_category     Accessories                             
country                   France United Kingdom United States
age_group                                                    
Adults (35-64)           762,773      1,034,101     3,260,448
Seniors (64+)              6,983         23,443        40,277
Young Adults (25-34)     610,806        615,722     1,766,303
Youth (<25)              247,127        277,734       752,295

In [8]:
df.query("country in ['France', 'United States', 'United Kingdom'] & product_category == 'Bikes'") \
    .pivot_table(
    index='age_group',
    columns=['product_category', 'country'],
    values='revenue',
    aggfunc='sum',
    fill_value=0,
    observed=True,
    sort=True
)

product_category         Bikes                             
country                 France United Kingdom United States
age_group                                                  
Adults (35-64)       3,103,937      3,944,222    12,288,385
Seniors (64+)                0         11,310        82,113
Young Adults (25-34) 3,507,960      3,057,910     7,085,938
Youth (<25)            766,452      1,171,226     2,095,061

In [9]:
df.query("country in ['France', 'United States', 'United Kingdom'] & product_category == 'Clothing'") \
    .pivot_table(
    index='age_group',
    columns=['product_category', 'country'],
    values='revenue',
    aggfunc='sum',
    fill_value=0,
    observed=True,
    sort=True
)

product_category     Clothing                             
country                France United Kingdom United States
age_group                                                 
Adults (35-64)        335,195        447,704     2,039,295
Seniors (64+)           9,751          5,598        21,019
Young Adults (25-34)  359,244        334,484       904,509
Youth (<25)           136,985        166,552       479,131